# Kernel Methods AMMI 2020
This is a data challenge for the course "Kernel Methods" for AMMI 2020

In [138]:
import numpy as np
import pandas as pd 
from tqdm import tqdm
import os 
import cvxopt
from cvxopt import matrix

###  Data loading and processing 

In [188]:
# Training features and label
trainx_df = pd.read_csv("Xtr.csv")
trainx_df_mat = pd.read_csv("Xtr_mat100.csv",header=None, sep=' ')

trainy_df = pd.read_csv("Ytr.csv")

# Test features and label
test_df = pd.read_csv("Xte.csv")
test_df_mat = pd.read_csv("Xte_mat100.csv",header=None, sep=' ')


In [189]:
trainx_df.head(5)

,Id,seq
0,0,GAGGGGCTGGGGAGGGGGCTGGCCCAGAGGCACCAGACTCTGCAGA...
1,1,CGGCCTGGGGGCCACATGTGAGTGCTTACCTGTGTGGGGATGAGGG...
2,2,GACAACGCCGCTGTCAGCCGCCTTCGACTCACCTGGGAGGTGATGA...
3,3,GCCTCCCTTGGCACCACGGGAGACCAGTTTTGGAGGGGCGGGGCTG...
4,4,GCACTACTACACCCATTGCTGTAATAGTAAGTGCCGGTGCCTTCAC...


In [190]:
trainx_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
Id     2000 non-null int64
seq    2000 non-null object
dtypes: int64(1), object(1)
memory usage: 31.4+ KB


In [191]:
trainy_df.head(5)

,Id,Bound
0,0,1
1,1,0
2,2,1
3,3,0
4,4,1


In [192]:
trainy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
Id       2000 non-null int64
Bound    2000 non-null int64
dtypes: int64(2)
memory usage: 31.4 KB


In [193]:
trainy_df.count()

Id       2000
Bound    2000
dtype: int64

In [194]:
print("Training shape",trainx_df.shape)
print("Label shape",trainy.shape)
print("Test shape",test_df.shape)

Training shape (2000, 2)
Label shape (2000,)
Test shape (1000, 2)


In [146]:
# Transformation dataframe into numpy
from sklearn import preprocessing
trainx = np.array(trainx_df)
trainx_mat = np.array(trainx_df_mat)
#trainx_mat = preprocessing.scale(trainx_mat, axis=1)

trainy = np.array(trainy_df)[:,1]

test = np.array(test_df)
test_mat = np.array(test_df_mat)
#test_mat = preprocessing.scale(test_mat,axis=1)


## Let's try with Gaussian Kernel

In [147]:
def sigma_from_median(X):
    '''
    Returns the median of ||Xi-Xj||
    
    Input
    -----
    X: (n, p) matrix
    '''
    pairwise_diff = X[:, :, None] - X[:, :, None].T
    pairwise_diff *= pairwise_diff
    euclidean_dist = np.sqrt(pairwise_diff.sum(axis=1))
    return np.median(euclidean_dist)
sigma = sigma_from_median(trainx_mat)

In [148]:
sigma

0.16268075594669312

In [149]:
## kernel function
def GaussKernel(X1, X2, sigma = 0.16268075594669312):
    n, m = X1.shape[0], X2.shape[0]
    K = np.zeros((n,m))
    
    for i in range(n):
        for j in range(m):
            K[i,j] = np.exp(-((np.linalg.norm(X1[i]-X2[j]))**2)/(2*sigma**2))
    return K

In [150]:
# kernel matrix
kernel_x = GaussKernel(trainx_mat,trainx_mat)

In [151]:
np.save("gaussian_kernel.npy",kernel_x)

### Let's implement SVM with Gaussian Kernel

We solve the optimization problem $$\left\{\begin{matrix}
\underset{\alpha \in \mathbb{R}^{n}}{\text{max}} \hspace{0.1cm} 2\alpha^{T}y - \alpha^{T}K\alpha \\ 0 \leq y_i\alpha_i \leq \frac{1}{2\lambda n}, \hspace{0.5cm} \text{for} \hspace{0.3cm} i = 0...n
\end{matrix}\right. \Leftrightarrow \left\{\begin{matrix}
\underset{\alpha \in \mathbb{R}^{n}}{\text{min}} \hspace{0.1cm} \frac{1}{2}\alpha^{T}P\alpha + q^{t}\alpha  \\ G\alpha \leq h
\end{matrix}\right.   $$
Where $\tilde{P} = K$, $q = -y$, $G =\binom{\text{Diag}(y)}{-\text{Diag}(y)} $ and $h=\binom{\frac{1}{2\lambda n}\mathcal{1}}{0}$

In [152]:
def solve_dual_SVM(K,y, lambda_ = 1):
    n = K.shape[0] 
    G = np.vstack((np.diag(y),-np.diag(y)))
    h = np.vstack(((1/(2*lambda_*n))*np.ones((n,1)),np.zeros((n,1))))

    P = K
    q = -y.reshape(-1,1)
    #P = .5 * (P + P.T)  # make sure P is symmetric
    args = [matrix(P), matrix(q)]

    args.extend([matrix(G), matrix(h)])

    sol = cvxopt.solvers.qp(*args) 

    return np.array(sol['x']).reshape((P.shape[1],))

In [153]:
alpha = solve_dual_SVM(kernel_x,2*trainy-1.,lambda_= 0.0000001)

     pcost       dcost       gap    pres   dres
 0:  2.0237e+08 -6.1851e+09  7e+09  8e-02  3e-11
 1:  1.0743e+08 -8.0930e+08  9e+08  1e-02  3e-11
 2:  4.4887e+07 -1.8142e+08  2e+08  2e-03  2e-11
 3:  1.5047e+07 -4.9446e+07  6e+07  3e-04  2e-11
 4:  5.3234e+06 -1.9840e+07  3e+07  6e-05  1e-11
 5:  1.5632e+06 -7.0446e+06  9e+06  2e-16  9e-12
 6:  2.5345e+05 -7.5716e+05  1e+06  2e-16  5e-12
 7:  1.1636e+04 -9.8212e+04  1e+05  2e-16  2e-12
 8: -1.6325e+04 -2.9916e+04  1e+04  2e-16  1e-12
 9: -1.8620e+04 -2.1623e+04  3e+03  2e-16  8e-13
10: -1.9049e+04 -1.9613e+04  6e+02  2e-16  8e-13
11: -1.9150e+04 -1.9202e+04  5e+01  2e-16  8e-13
12: -1.9163e+04 -1.9165e+04  2e+00  2e-16  8e-13
13: -1.9164e+04 -1.9164e+04  7e-02  2e-16  8e-13
14: -1.9164e+04 -1.9164e+04  3e-03  2e-16  8e-13
Optimal solution found.


### Make prediction on test data

In [154]:
# kernel matrix test
kernel_test = GaussKernel(trainx_mat,test_mat)

In [155]:
np.save("gaussian_kernel_test.npy",kernel_test)

In [156]:
prediction = alpha.reshape(-1,1).T.dot(kernel_test)
prediction[prediction>0] = 1
prediction[prediction <0] = 0

### Evaluate the performance of the model

In [157]:
train_prediction = (np.sign(alpha.reshape(-1,1).T.dot(kernel_x))+1)/2
print('Train Accuracy :',1- np.abs(train_prediction - trainy).sum()/trainy.shape[0])

Train Accuracy : 1.0


### Results

In [158]:
predictions = np.squeeze(prediction).astype(int)
df = pd.DataFrame({'Bound': predictions,
                   'Id': np.arange(len(predictions))})
df = df[['Id','Bound']]

In [159]:
df.to_csv("gaussian_SVM.csv",index = False)

# Spectrum Kernel

## kernel functions

In [160]:
def getSubString(mString, spectrum):
    
    dictionnary = {}
    for i in range(len(mString)-spectrum+1):
        if mString[i:i+spectrum] in dictionnary:
            dictionnary[mString[i:i+spectrum]] += 1
        else:
            dictionnary[mString[i:i+spectrum]] = 1
    return dictionnary

def SpectrumKernelFunction(mString1, mString2, spectrum):
    dictionnary1 = getSubString(mString1, spectrum)
    dictionnary2 = getSubString(mString2, spectrum)
    
    kernel = 0
    for i in dictionnary1:
        if i in dictionnary2:
            kernel += dictionnary1[i] * dictionnary2[i]
    return kernel

## We should improve this function to take less time
def SpectrumKernelMatrix_train(serie,spectrum):
    n = serie.shape[0]
    K = np.zeros((n,n))
    for i,seq1 in enumerate(serie):
        for j,seq2 in enumerate(serie):
            if i <= j :
                K[i,j] = SpectrumKernelFunction(seq1, seq2, spectrum)
                K[j,i] = K[i,j]
    return(K)

def SpectrumKernelMatrix_test(serie_train, serie_test, spectrum):
    n = serie_train.shape[0]
    m = serie_test.shape[0]
    K = np.zeros((n,m))
    for i,seq1 in enumerate(serie_test):
        for j,seq2 in enumerate(serie_train):
            K[j,i] = SpectrumKernelFunction(seq1, seq2, spectrum)
    return(K)
    

In [161]:
# Paralize computation and kernel matrix contruction for training set

if os.path.isfile("spectrum_kernel_Xtr.npy"):
    K_Xtr = np.load("spectrum_kernel_Xtr.npy")
else:
    K_Xtr = SpectrumKernelMatrix_train(trainx_df['seq'],spectrum=3)
    np.save("spectrum_kernel_Xtr.npy",K_Xtr)

In [162]:
# Paralize computation and kernel matrix contruction for test set
if os.path.isfile("spectrum_kernel_Xte.npy"):
    K_Xte = np.load("spectrum_kernel_Xte.npy")
else:
    K_Xte = SpectrumKernelMatrix_test(trainx_df['seq'],test_df['seq'],spectrum=3)
    np.save("spectrum_kernel_Xte.npy",K_Xte)

### 1.2. Solve the standard weighted kernel logisitc regression (WKLR) problem

In [163]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

### We need to improve this ####
def sqrtMatrix(W):
    # To compute the square root of a symetric positive matrix
    D,V = np.linalg.eig(W)
    return np.dot(np.dot(V,np.diag(np.sqrt(D))),np.linalg.inv(V))

def solveWKRR(K,W,z,lambda_):
    n = K.shape[0]
    W_sqrt = np.real(sqrtMatrix(W))
    
    temp = np.dot(np.dot(W_sqrt,K),W_sqrt) +  n*lambda_*np.eye(n)
    return  np.dot(W_sqrt,np.linalg.solve(temp,np.dot(W_sqrt,z)))

def solveKLR(K,y,alpha0,lambda_ = 0.00000001,itermax = 30, eps =1e-6):
    n = K.shape[0]
    
    iter_ = 0
    last_alpha = 10*alpha0 + np.ones(alpha0.shape)
    alpha = alpha0
    
    while (iter_< itermax) and (np.linalg.norm(last_alpha-alpha)>eps) :         
        print(iter_,np.linalg.norm(last_alpha-alpha))
        last_alpha = alpha
        m = np.dot(K,alpha)
        P = np.zeros((n,1))
        W = np.zeros((n,n))
        z = np.zeros((n,1))
        for i in range(n):
            P[i,0] = -sigmoid(-y[i]*m[i])
            W[i,i] = sigmoid(m[i])*sigmoid(-m[i])
            z[i,0] = m[i] - (P[i,0]*y[i])/W[i,i]
        alpha = solveWKRR(K,W,z,lambda_)
        iter_ = iter_ +1
          
    return alpha        

In [164]:
K0 = K_Xtr
y_0 = trainy.reshape((trainy.shape[0],1))
y_0 = 2*y_0-1
n = y_0.shape[0]
alpha = np.zeros((n,1))
alpha = solveKLR(K0,y_0,alpha,10) 


0 44.721359549995796
1 0.0011119556177038219


In [165]:
def sign(x):
    y = x
    n = x.shape[0]
    for i in range(n):
        if x[i,0] > 0:
            y[i,0] = 1
        else:
            y[i,0] = 0
    return y

print('Accuracy:',np.linalg.norm(1-sign(np.dot(K0,alpha))+y_0,1)/y_0.shape[0])

Accuracy: 0.918


## Results

In [166]:
prediction2 = alpha.reshape(-1,1).T.dot(K_Xte)
prediction2[prediction2>0] = 1
prediction2[prediction2 <0] = 0

In [167]:
predictions = np.squeeze(prediction2).astype(int)
print(len(predictions))
df = pd.DataFrame({'Bound': predictions,
                   'Id': np.arange(len(predictions))})
df = df[['Id','Bound']]

1000


In [168]:
df.to_csv("spetrum_SVM.csv",index = False)

## tf_idf

## spectrum_embedding

In [169]:
import itertools

def k_mers(Alphabet, k):
    'Compute all possible words of size k from Alphabet'
    'Store the result as a dictionnary where the keys are the words and the values ar integer Ids'
    all_kmers_tuple = list(itertools.product(Alphabet, repeat=k))
    all_kmers = list(map(lambda tup: ''.join(tup), all_kmers_tuple))
    return dict(zip(all_kmers, range(len(all_kmers))))

    
def spectrum_embedding(sequence, all_kmers_dict, k):
    'Compute the k-sepctrum embedding of sequence'
    'The result is a vector of size vocabulary'
    embedding = np.zeros(len(all_kmers_dict))
    for idx in range(len(sequence)-k+1): # slidding window of size k on the sequence
        word_id = all_kmers_dict[sequence[idx:idx+k]]
        embedding[word_id] += 1  
    return(embedding)

def data_embedding(df, all_kmers_dict, k):
    nb_sequences = df.shape[0]
    embedding_dict = dict.fromkeys(range(nb_sequences))
    for idx,sequence in enumerate(list(df['seq'])):
        embedding = spectrum_embedding(sequence, all_kmers_dict, k)
        embedding_dict[idx] = embedding  
    return embedding_dict
    
        

### Compute the embeddings for all data sets 

In [180]:
k = 3
Alphabet = ['G', 'C', 'A', 'T']

all_kmers_dict = k_mers(Alphabet, k)


Xtr_embedding = data_embedding(trainx_df,all_kmers_dict,k)

Xte_embedding = data_embedding(test_df,all_kmers_dict,k)

### Computing tf_idf scores

In [181]:
def counting_array(embedding_dict, all_kmers_dict):
    'Compute the count matrix of kmer by sequence'
    'Output is an array of size nb_seq*vocab_size'
    D = len(embedding_dict) # Number of documents
    T = len(all_kmers_dict) # Vocabulary size
    
    output = np.zeros((D,T))
    for document in embedding_dict.keys():
        output[document] = embedding_dict[document]
    return output

def tf_idf(D):
    'Input : D is a conting matrix (nb_seq*vocab_size)'
    'Ouptut: array of tf_idf socres'
    N = D.shape[0] # number of documents
    idf = np.log(N/np.count_nonzero(D,axis=1)).reshape(-1,1)
    tf = np.log(1+D/np.sum(D,axis = 1).reshape(-1,1))
    return tf*idf

Compute the tf_idf for each dataset

In [182]:
# Tf-Idf numpy.arrays -- train
D_tr = counting_array(Xtr_embedding, all_kmers_dict)
Xtr = tf_idf(D_tr)


# Tf-Idf numpy.arrays -- test
D_te = counting_array(Xte_embedding, all_kmers_dict)
Xte = tf_idf(D_te)


# Transforming the labels into numpy.arrays 
y0 = 2*np.array(trainy_df)[:,1] - 1

In [183]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler().fit(Xtr)
Xtr = sc.transform(Xtr)
Xte = sc.transform(Xte)

 ## SVM + Guaussian on tf_idf embeddings

In [184]:
# We should parallelize this computation
if os.path.isfile("tf_idf_kernel_Xtr.npy"):
    K_Xtr = np.load("tf_idf_kernel_Xtr.npy")
else:
    K_Xtr = GaussKernel(Xtr, Xtr, sigma = 5)
    np.save("tf_idf_kernel_Xtr.npy",K_Xtr)
    
    
if os.path.isfile("tf_idf_kernel_Xte.npy"):
    K_Xte = np.load("tf_idf_kernel_Xte.npy")
else:
    K_Xte = GaussKernel(Xtr, Xte, sigma = 5)
    np.save("tf_idf_kernel_Xte.npy",K_Xte)
 

# Transforming the labels into numpy.arrays 
y = 2*np.array(trainy_df)[:,1] - 1.

In [185]:
alpha_star = solve_dual_SVM(K_Xtr,y, lambda_= 1e-8)

     pcost       dcost       gap    pres   dres
 0:  4.9372e+10 -5.2964e+11  6e+11  5e-02  4e-11
 1:  2.4374e+10 -6.5928e+10  9e+10  6e-03  2e-10
 2:  8.0141e+09 -2.0013e+10  3e+10  1e-03  4e-11
 3:  1.5651e+09 -3.6132e+09  5e+09  1e-16  2e-11
 4:  2.6005e+08 -3.5150e+08  6e+08  2e-16  9e-12
 5:  3.7492e+07 -4.2871e+07  8e+07  2e-16  3e-12
 6:  5.3262e+06 -6.1494e+06  1e+07  2e-16  1e-12
 7:  7.3911e+05 -8.9905e+05  2e+06  2e-16  5e-13
 8:  9.5832e+04 -1.3834e+05  2e+05  2e-16  2e-13
 9:  9.1912e+03 -2.4019e+04  3e+04  2e-16  8e-14
10: -1.1970e+03 -5.6385e+03  4e+03  2e-16  3e-14
11: -2.0622e+03 -2.6634e+03  6e+02  2e-16  2e-14
12: -2.1258e+03 -2.2292e+03  1e+02  2e-16  1e-14
13: -2.1359e+03 -2.1457e+03  1e+01  2e-16  1e-14
14: -2.1372e+03 -2.1377e+03  5e-01  2e-16  1e-14
15: -2.1373e+03 -2.1373e+03  2e-02  2e-16  1e-14
16: -2.1373e+03 -2.1373e+03  9e-04  2e-16  1e-14
Optimal solution found.


In [186]:
prediction = alpha_star.reshape(-1,1).T.dot(K_Xte)
prediction[prediction>0] = 1
prediction[prediction <0] = 0


## Evaluation

In [187]:
train_prediction = (np.sign(alpha_star.reshape(-1,1).T.dot(K_Xtr))+1)/2
print('Train Accuracy :',1- np.abs(train_prediction - y).sum()/y.shape[0])

Train Accuracy : 0.499


## Writing the result

In [178]:
predictions = np.squeeze(prediction).astype(int)
df = pd.DataFrame({'Bound': predictions,
                   'Id': np.arange(len(predictions))})
df = df[['Id','Bound']]

df.to_csv("tf_idf_SVM.csv",index = False)